### Using averagine modeled reference patterns for charge state deconvolution

Translating slices of mass spectra into a set of representative keys is not limited to self-similarity detection. Other approaches used LSH for fast candidate identification of fragment-spectra, see for example Wang et al.[^1][^2]. Here, a look-up database is built by first mapping all reference sepctra into LSH-key representations. Using LSH first, unidentified candidate spectra only need to be matched with a subset of candidates in the database, greatly reducing the total number of comparisons. For a more detailed explanation of algorithmic solutions to spectral identification in general, the interested reader is refered to Edward M. Marcotte[^3].

We built on the idea of a pre-computed reference, but used typical isotope patterns to find peptide ions of a specific charge state in precursor spectra instead. These are hashed to form a reference database, which can then be used to detect isotope patterns in sets of spectrum windows directly. This approach is of course more restrictive when it comes to the space of allowed signal shapes, as it requires a window to be similar to one of the modeled peak distributions.

The averagine model as proposed by Senko et al.[^4] was used to create all reference patterns:

\begin{equation}
I(x \rvert m, z, \sigma) = \sum_{k=0}^{K} w(m, k) \, \mathcal{N}\left(x \bigg| \frac{m + k\mathrm{m_n}}{z}, \sigma^{2} \right)
\end{equation}

\begin{equation}
w(m,k) = \mathrm{e}^{-\lambda(m)}\frac{\lambda^k(m)}{\Gamma(k+1)}
\end{equation}

\begin{equation}
\lambda(m) = \beta m + \gamma
\end{equation}

where $x$ denotes position on the $m/z$ axis, $m$ the mass of the molecule, $z$ the charge state of the molecule, $\sigma$ the width of the peak, $\mathcal{N}$ probability density function of the Gaussian distribution, $\mathrm{m_n}$ mass of the neutron, $\Gamma$ the gamma function, $\beta$ = $-3.091 \cdot 10^{-2}$ and $\gamma = 5.94 \cdot 10^{-4} \mathrm{Da}^{-1}$. 

It was sampled with a step size of $10^{-5}$ in a range from $-1$ to $+9$ Da around the monoisotopic peak and afterwards binned to a resolution of $10^{-2}$ Da. This was done for all masses from 150 to 1800 Da and for all charge states from 1 to 5.

We then applied our hashing strategy to those synthetical mass spectra the same way it was applied to self-similarity detection. When an unidentified window is now hashed, its key-set can be used to select a set of candidate isotope-patterns, for which cosine similarity is calculated explicitely. Should a certain threshold of similarity be surpassed, the highest scoring reference pattern is returned together with its charge state and monoisotopic mass.

---
[^1]: msCRUSH: Fast Tandem Mass Spectral Clustering Using Locality Sensitive Hashing.
journal of proteome, 2019. https://pubs.acs.org/doi/10.1021/acs.jproteome.8b00448

[^2]: A Fast and Memory-Efficient Spectral Library Search Algorithm Using Locality-Sensitive Hashing. 
Proteomics, 2020.  https://doi.org/10.1002/pmic.202000002

[^3]: How do shotgun proteomics algorithms identify proteins?
nature biotechnology, 2007. https://doi.org/10.1038/nbt0707-755

[^4]: Senko et al.

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

import time
from tqdm.notebook import tqdm

from proteolizarddata.data import PyTimsDataHandle, TimsFrame, MzSpectrum
from proteolizardalgo.hashing import TimsHasher, IsotopeReferenceSearch, ReferencePattern
from proteolizardalgo.utility import create_reference_dict, get_refspec_list, get_ref_pattern_as_spectra

import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from sklearn.neighbors import KDTree

In [2]:
dh = PyTimsDataHandle('../data/M210115_001_Slot1-1_1_850.d/')

In [3]:
ref_hashes = pd.read_parquet('synthetic_patterns_last_contrib.parquet')
d_list = get_refspec_list(get_ref_pattern_as_spectra(ref_hashes), win_len=4)
d = create_reference_dict(d_list)

creating reference patterns: 100%|███████████████████████████| 17857/17857 [01:59<00:00, 149.89it/s]


In [51]:
hasher = TimsHasher(trials=512, len_trial=22, seed=42, num_dalton=4, resolution=2)

In [52]:
ref_search = IsotopeReferenceSearch(d, hasher=hasher)

calculating keys: 100%|███████████████████████████████████████████| 389/389 [00:16<00:00, 23.67it/s]


In [59]:
dda_precursors = dh.get_selected_precursors()
all_parent_ids = np.random.choice(sorted(list(set(dda_precursors.Parent))), 250)

In [60]:
t_list = []

for i in tqdm(all_parent_ids):

    random_id = i
    
    try:
        frame = dh.get_frame(random_id)
        patterns = ref_search.find_isotope_patterns(frame, min_intensity=100, min_peaks=4, overlapping=True)

        prec = dda_precursors[dda_precursors.Parent == random_id].sort_values('MonoisotopicMz').dropna()
        prec['MonoisotopicMz'] = np.round(prec['MonoisotopicMz'], 2)
        prec['ScanNumber'] = np.round(prec['ScanNumber'])

        kdt = KDTree(patterns[['scan', 'charge', 'mz_mono']].values, leaf_size=30, metric='manhattan')
        distance, index = kdt.query(prec[['ScanNumber', 'Charge', 'MonoisotopicMz']].values)

        matched = pd.DataFrame(np.hstack((np.squeeze(patterns[['scan', 'charge', 'mz_mono']].values[index]), 
                                prec[['MonoisotopicMz', 'ScanNumber', 'Charge']].values, distance)), 

                               columns=['scan', 'charge', 'mz_mono', 
                 'MonoisotopicMz', 'ScanNumber', 'Charge', 'distance']).sort_values(['distance'])

        matched['mz_diff'] = np.round(np.abs(matched['mz_mono'] - matched['MonoisotopicMz']), 2)

        t_list.append(matched)
        
    except Exception as e:
        print(e)
        pass

  0%|          | 0/250 [00:00<?, ?it/s]

all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)
all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)


In [117]:
alle = pd.concat(t_list).sort_values(['charge', 'mz_diff'])
alle['mz-diff-mod'] = (alle['mz_diff'] * alle['charge']) % 1
alle

,scan,charge,mz_mono,MonoisotopicMz,ScanNumber,Charge,distance,mz_diff,mz-diff-mod
18,292.0,1.0,1229.62,1229.62,289.0,2.0,4.00,0.00,0.00
1,779.0,1.0,381.72,381.72,779.0,2.0,1.00,0.00,0.00
11,565.0,1.0,622.03,622.03,567.0,1.0,2.00,0.00,0.00
4,781.0,1.0,413.15,413.15,781.0,2.0,1.00,0.00,0.00
18,653.0,1.0,623.31,623.31,653.0,2.0,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...
7,526.0,5.0,930.21,930.61,526.0,5.0,0.40,0.40,0.00
6,482.0,5.0,878.05,878.45,481.0,5.0,1.40,0.40,0.00
25,697.0,5.0,600.09,599.69,695.0,5.0,2.40,0.40,0.00
3,521.0,5.0,893.62,890.11,514.0,3.0,12.51,3.51,0.55


In [94]:
def calculate_matches(match_table, allowed_total_diff=6.0):
    
    inside_diff = match_table[match_table['distance'] <= allowed_total_diff]
    diff_and_charge = inside_diff[inside_diff['mz-diff-mod'] <= 0.02]
    
    return diff_and_charge

In [123]:
alle_diff = alle[alle['distance'] <= 10]
alle_diff

,scan,charge,mz_mono,MonoisotopicMz,ScanNumber,Charge,distance,mz_diff,mz-diff-mod
18,292.0,1.0,1229.62,1229.62,289.0,2.0,4.00,0.00,0.00
1,779.0,1.0,381.72,381.72,779.0,2.0,1.00,0.00,0.00
11,565.0,1.0,622.03,622.03,567.0,1.0,2.00,0.00,0.00
4,781.0,1.0,413.15,413.15,781.0,2.0,1.00,0.00,0.00
18,653.0,1.0,623.31,623.31,653.0,2.0,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...
9,596.0,5.0,796.19,796.39,598.0,5.0,2.20,0.20,0.00
31,591.0,5.0,838.79,838.58,596.0,5.0,5.21,0.21,0.05
7,526.0,5.0,930.21,930.61,526.0,5.0,0.40,0.40,0.00
6,482.0,5.0,878.05,878.45,481.0,5.0,1.40,0.40,0.00


In [124]:
alle_diff_charge = alle_diff[alle_diff.charge == alle_diff.Charge].sort_values(['distance'])
alle_diff_charge

,scan,charge,mz_mono,MonoisotopicMz,ScanNumber,Charge,distance,mz_diff,mz-diff-mod
17,488.0,2.0,853.43,853.43,488.0,2.0,0.00,0.00,0.00
6,650.0,3.0,586.30,586.30,650.0,3.0,0.00,0.00,0.00
0,709.0,2.0,512.73,512.73,709.0,2.0,0.00,0.00,0.00
20,479.0,2.0,874.89,874.89,479.0,2.0,0.00,0.00,0.00
16,506.0,2.0,809.91,809.91,506.0,2.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...
14,464.0,2.0,933.94,930.00,458.0,2.0,9.94,3.94,0.88
2,508.0,2.0,817.91,826.90,507.0,2.0,9.99,8.99,0.98
18,379.0,2.0,1111.56,1111.56,369.0,2.0,10.00,0.00,0.00
20,444.0,3.0,1196.24,1196.24,434.0,3.0,10.00,0.00,0.00


In [127]:
alle_diff_charge_mono = alle_diff_charge[alle_diff_charge['mz_diff'] <= 0.02]
alle_diff_charge_mono

,scan,charge,mz_mono,MonoisotopicMz,ScanNumber,Charge,distance,mz_diff,mz-diff-mod
17,488.0,2.0,853.43,853.43,488.0,2.0,0.00,0.00,0.00
6,650.0,3.0,586.30,586.30,650.0,3.0,0.00,0.00,0.00
0,709.0,2.0,512.73,512.73,709.0,2.0,0.00,0.00,0.00
20,479.0,2.0,874.89,874.89,479.0,2.0,0.00,0.00,0.00
16,506.0,2.0,809.91,809.91,506.0,2.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...
24,446.0,2.0,899.97,899.98,437.0,2.0,9.01,0.01,0.02
48,508.0,1.0,808.46,808.47,499.0,1.0,9.01,0.01,0.01
18,379.0,2.0,1111.56,1111.56,369.0,2.0,10.00,0.00,0.00
20,444.0,3.0,1196.24,1196.24,434.0,3.0,10.00,0.00,0.00


### Results